# Python装饰器

**装饰器本质上是一个 Python 函数或类**，它可以让其他函数或类在不需要做任何代码修改的前提下增加额外功能，装饰器的返回值也是一个函数/类对象。它经常用于有切面需求的场景，比如：**插入日志、性能测试、事务处理、缓存、权限校验等场景**，装饰器是解决这类问题的绝佳设计。有了装饰器，我们就可以抽离出大量与函数功能本身无关的雷同代码到装饰器中并继续重用。概括的讲，**装饰器的作用就是为已经存在的对象添加额外的功能**。

## 1.引言（Python函数）

Python 中的函数和 Java、C++不太一样，**Python 中的函数可以像普通变量一样当做参数传递给另外一个函数**。

In [19]:
def test():
    print("this is what I want")

def receive(func):
    print("I receive a function")
    func()

receive(test)

I receive a function
this is what I want


>装饰器在 Python 使用如此方便都要归因于 Python 的函数能像普通的对象一样能
1.**作为参数**传递给其他函数，可以被赋值给其他变量
2.**可以作为返回值**
3.可以被**定义在另外一个函数内**。

## 2.简单理解装饰器

一个**简答例子**帮助理解

**（1）一个简单的业务函数如下：**

In [2]:
def test():
    print("this is what I want")
test()

this is what I want


**（2）后来需要加一个新需求**

In [3]:
def test():
    print("call test()")  #新需求
    print("this is what I want")
test()

call test()
this is what I want


>如果函数 test()、test2()……等一大组函数也有类似的需求，需要在这些函数里重复写新需求，造成问题：
1.修改工作量大
2.造成大量雷同的代码
3.不利于之后的业务扩展

**（3）一种简单方法是用引言的方法，实现如下：**

In [1]:
def log(func):
    print("call test()") 
    func()

def test():
    print("this is what I want")

log(test)

call test()
this is what I want


>但是可以简单看出该方法的问题:
上面的修改实现了功能（不改变业务函数，加入新功能）,但是**调用的时候不再是调用真正的业务函数test()，而是调用辅助函数log()**。

**（4）引出了简单装饰器的方法**

In [16]:
def log(func):
    def wrapper():
        print("call test()") 
        func()
    return wrapper

def test():
    print("this is what I want")

# 因为装饰器 use_logging(test) 返回的是函数对象 wrapper
# 这条语句相当于  test = wrapper
test= log(test) 
print(test)
test()  # 执行test() 相当于执行 wrapper()

<function log.<locals>.wrapper at 0x000001510FACCBF8>
call test()
this is what I want


>辅助函数log()就是一个装饰器，它把执行真正业务函数 func 包裹在其中，看起来像 test()被log()装饰了一样。
**这么做可以实现面向切面编程AOP**。

**（4）引出 @ 语法糖来实现装饰器**

对上面程序进行改写

In [ ]:
def log(func):
    def wrapper():
        print("call test()")
        func()
    return wrapper

@log
def test():
    print("this is what I want")

test()

@log实现了test= log(test) 的功能。
>如此一来，业务函数test()不需要做任何修改，只需在定义的地方加上装饰器，调用的时候不用加任何修饰。
这样，我们就提高了程序的可重复利用性，并增加了程序的可读性。

## 3.带参数的业务函数

**（1）首先回顾包裹传递法。**

In [18]:
def foo(*args, **kwargs):
    print('args = ', args)
    print('kwargs = ', kwargs)
    print('---------------------------------------')

if __name__ == '__main__':
    foo(1,2,3,4)
    foo(a=1,b=2,c=3)
    foo(1,2,3,4, a=1,b=2,c=3)
    foo('a', 1, None, a=1, b='2', c=3)

args =  (1, 2, 3, 4)
kwargs =  {}
---------------------------------------
args =  ()
kwargs =  {'a': 1, 'b': 2, 'c': 3}
---------------------------------------
args =  (1, 2, 3, 4)
kwargs =  {'a': 1, 'b': 2, 'c': 3}
---------------------------------------
args =  ('a', 1, None)
kwargs =  {'a': 1, 'b': '2', 'c': 3}
---------------------------------------


>\*args表示任何多个无名参数，它是一个tuple
\*\*kwargs表示关键字参数，它是一个dict
同时使用\*args和\*\*kwargs时，必须\*args参数列要在\*\*kwargs前

**（2）带有有限个参数**

当业务函数test()需要参数，比如 test(a,b)，实现如下：

In [19]:
def log(func):
    def wrapper(a,b):
        print("call test(%d，%d)" %(a,b))
        func(a,b)
    return wrapper

@log
def test(a,b):
    print("sum = %d" % (a+b))

test(2,4)

call test(2，4)
sum = 6


**（3）带有可变数量的参数**

当装饰器不知道业务函数到底有多少个参数时，用\*args 来代替

In [20]:
def log(func):
    def wrapper(*args):
        print("call test()" )
        func(*args)
    return wrapper

@log
def test(a,b,c):
    print("sum = %d" % (a+b+c))

test(2,4,5)

call test()
sum = 11


**（4）含关键字的可变参数**

In [1]:
def log(func):
    def wrapper(*args, **kwargs):
        print("call test()" )
        func(*args, **kwargs)
    return wrapper

@log
def test(a,b,c,way=None):
    print("sum = %d,way = %s" % ((a+b+c),way))

test(2,4,5,"add")

call test()
sum = 11,way = add


## 4.带参数的装饰器

装饰器还有更大的灵活性，例如带参数的装饰器，在上面的装饰器调用中，该装饰器接收唯一的参数就是业务函数 test()。




装饰器的语法允许我们在调用时，提供其它参数，比如@decorator(a)。这样，就为装饰器的编写和使用提供了更大的灵活性。比如，我们可以在装饰器中指定日志的等级，因为不同业务函数可能需要的日志级别是不一样的。

In [5]:
def log(level):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if level == "warn":
                print("%s waring" % func.__name__)
            elif level == "info":
                print("%s infomation get" % func.__name__)
            return func(*args, **kwargs)
        return wrapper
    return decorator


@log(level="warn")
def test():
    print("this is what I want")

@log(level="info")
def test2():
    print("this is also what I want")

test()
test2()

test waring
this is what I want
test2 infomation get
this is also what I want


上面的 log() 是允许带参数的装饰器。它实际上是对原有装饰器的一个函数封装，并返回一个装饰器。我们可以将它理解为一个含有参数的闭包。当我们用@log(level="warn")调用的时候，Python 能够发现这一层的封装，并把参数传递到装饰器的环境中。

## 5.类装饰器

装饰器不仅可以是函数，还可以是类；相比函数装饰器，类装饰器具有灵活度大、高内聚、封装性等优点。

>**用法:**
1.让类的构造函数__init__()接受一个函数；
2.**重载**__call__()**并 返回一个函数**；
3.使用**@类**形式将装饰器附加到业务函数上。

**例子1：**

In [3]:
class DecorateDemo(object):
    def  __init__(self, func):
        self.__func = func
    def  __call__(self):
        print("before class decorator")
        self.__func()
        print("after class decorator")
        return self.__func

@DecorateDemo
def test():
    print("this is what I want")

test()

before class decorator
this is what I want
after class decorator


<function __main__.test>

**例子2：**

In [4]:
class myDecorator(object):
    def __init__(self, f):
        print("inside myDecorator.__init__()")
        f() # Prove that function definition has completed
    def __call__(self):
        print("inside myDecorator.__call__()")

@myDecorator
def aFunction():
    print("inside aFunction()")

inside myDecorator.__init__()
inside aFunction()


>可以看到在函数定义阶段，类初始化和自己函数的执行部分已经完成了

In [5]:
print("Finished decorating aFunction()")

aFunction()

Finished decorating aFunction()
inside myDecorator.__call__()


>可以看到在函数执行阶段，去执行了类的call

## 6.functools模块

functools模块提供了两个装饰器，主要用 functools.wraps。

使用装饰器极大地复用了代码，但是他有一个缺点就是原函数的元信息不见了。

**上面的方法（不用functools.wraps）**

In [7]:
def log(func):
    def wrapper():
        print("call test()")
        return func()
    return wrapper

@log
def test():
    print("this is what I want")

test()
print(test.__name__)

call test()
this is what I want
wrapper


**加入functools模块的方法**

In [9]:
import functools

def log(func):
    @functools.wraps(func)
    def wrapper():
        print("call test()")
        return func()
    return wrapper

@log
def test():
    print("this is what I want")

test()
print(test.__name__)

call test()
this is what I want
test


## 7. 多个装饰器

一个函数还可以同时定义多个装饰器，比如：

In [ ]:
@a
@b
@c
def f ():
    pass

在上面的函数定义阶段，装饰器执行顺序是从里到外，最先调用最里层的装饰器，最后调用最外层的装饰器，即c->b->a，它等效于

In [ ]:
f = a(b(c(f)))

因为a、b、c的返回值都是一个函数，所以a(b(c(f)))也是一个函数。执行函数f()相当于a(b(c(f)))(),它的执行顺序为a->b->c->f->c->b->a。

>总的来的，f()的结果为c->b->a->a->b->c->f->c->b->a

>即：装饰器函数的执行顺序是**分为（被装饰函数）定义阶段和（被装饰函数）执行阶段**的，装饰器函数在被装饰函数定义好后立即执行：

>1.在函数定义阶段：执行顺序是从最靠近函数的装饰器开始，自内而外的执行

>2.在函数执行阶段：执行顺序由外而内，一层层执行

**例子(看例子更清楚)：**

In [1]:
def log(func):
    print("log")
    def wrapper():
        print("call test()")
        result = func()
        print("end test()")
        return result 
    return wrapper

def hello(func):
    print("hello")
    def wrapper():
        print("hello test()")
        result = func()
        print("goodbye test()")
        return result 
    return wrapper

@log
@hello
def test():
    print("this is what I want")

# test()

hello
log


>结论：装饰器函数在被装饰函数定义好后立即执行 并且是从下往上执行。

In [2]:
test()

call test()
hello test()
this is what I want
goodbye test()
end test()


>结论：执行函数在执行的时候，顺序为log->hello->test->hello->log

## 8. 使用@property

### 8.1 引言

在绑定属性时，如果我们直接把属性暴露出去，虽然写起来很简单，但是，没办法检查参数，导致可以把成绩随便改：

In [ ]:
s = Student()
s.score = 9999

这显然不合逻辑。为了限制score的范围，可以通过一个**set_score()** 方法来设置成绩，再通过一个**get_score()** 来获取成绩，这样，在set_score()方法里，就可以检查参数：

In [4]:
class Student(object):

    def get_score(self):
         return self._score

    def set_score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value


In [5]:
s = Student()
s.set_score(60) # ok!
print(s.get_score())
s.set_score(9999)

60


ValueError: score must between 0 ~ 100!

### 8.2 @property

1.把一个方法变成属性，只需要加上@property就可以了，此时，@property本身又创建了另一个装饰器@score.setter，负责把一个setter方法变成属性赋值，于是，我们就拥有一个可控的属性操作：

In [6]:
class Student(object):
 
    @property
    def score(self):
        return self._score
#  @property本身又创建了另一个装饰器@score.setter，负责把一个setter方法变成属性赋值
    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value

In [7]:
s = Student()
s.score = 60 # OK，实际转化为s.set_score(60)
print(s.score)
s.score = 9999
print(s.score)

60


ValueError: score must between 0 ~ 100!

2.还可以定义只读属性，只定义getter方法，不定义setter方法就是一个只读属性:

In [ ]:
class Student(object):

    @property
    def birth(self):
        return self._birth

    @birth.setter
    def birth(self, value):
        self._birth = value

    @property
    def age(self):
        return 2015 - self._birth

>上面的birth是可读写属性，而age就是一个只读属性，因为age可以根据birth和当前时间计算出来。